# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

/home/iss-user/anaconda3/envs/iss-env-py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:
sys.path

['',
 '/home/iss-user/anaconda3/envs/iss-env-py3/lib/python36.zip',
 '/home/iss-user/anaconda3/envs/iss-env-py3/lib/python3.6',
 '/home/iss-user/anaconda3/envs/iss-env-py3/lib/python3.6/lib-dynload',
 '/home/iss-user/anaconda3/envs/iss-env-py3/lib/python3.6/site-packages',
 '/home/iss-user/anaconda3/envs/iss-env-py3/lib/python3.6/site-packages/mujoco_py-1.50.1.36-py3.6.egg',
 '/home/iss-user/anaconda3/envs/iss-env-py3/lib/python3.6/site-packages/torchvision-0.2.0-py3.6.egg',
 '/home/iss-user/anaconda3/envs/iss-env-py3/lib/python3.6/site-packages/IPython/extensions',
 '/home/iss-user/.ipython',
 '..']

## Object detection imports
Here are the imports from the object detection module.

In [4]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

In [5]:
# What model to download.

# MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017' # 30 21
# MODEL_NAME = 'mask_rcnn_inception_v2_coco_2018_01_28' # 79 25
MODEL_NAME = 'faster_rcnn_nas_coco_2018_01_28' # 1833 43
# MODEL_NAME = ''

# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_oid_2018_01_28' 
# MODEL_NAME = ''

# MODEL_NAME = 'faster_rcnn_resnet101_kitti_2018_01_28'
# MODEL_NAME = ''


MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
# PATH_TO_LABELS = os.path.join('data', 'openimage_label_map.pbtxt')

NUM_CLASSES = 90
# NUM_CLASSES = 19995

## Download Model

In [ ]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
'''
# For OpenImage's pre-trained model only: MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_oid_2018_01_28'
# 2018-02-23
# load OpenImage's 19995 different object classes
import csv

reader = csv.reader(open('data/class-descriptions.csv', 'r'))
dict_csv = []
dict_object_class_list = []
dict_object_class = {}

for line in reader:
    dict_csv.append(line)

for i in range(0, len(dict_csv)):
    dict_object_class = {}
    dict_object_class['id'] = i+1
    dict_object_class['name'] = dict_csv[i][1] #     dict_object_class['display_name'] = dict_csv[i][1]
    dict_object_class_list.append(dict_object_class)

categories = dict_object_class_list
category_index = label_map_util.create_category_index(categories)

NUM_CLASSES = len(categories)

categories[0:10]
'''

"\n# For OpenImage's pre-trained model only: MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_oid_2018_01_28'\n# 2018-02-23\n# load OpenImage's 19995 different object classes\nimport csv\n\nreader = csv.reader(open('data/class-descriptions.csv', 'r'))\ndict_csv = []\ndict_object_class_list = []\ndict_object_class = {}\n\nfor line in reader:\n    dict_csv.append(line)\n\nfor i in range(0, len(dict_csv)):\n    dict_object_class = {}\n    dict_object_class['id'] = i+1\n    dict_object_class['name'] = dict_csv[i][1] #     dict_object_class['display_name'] = dict_csv[i][1]\n    dict_object_class_list.append(dict_object_class)\n\ncategories = dict_object_class_list\ncategory_index = label_map_util.create_category_index(categories)\n\nNUM_CLASSES = len(categories)\n\ncategories[0:10]\n"

---

In [8]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [9]:
categories

[{'id': 1, 'name': 'person'},
 {'id': 2, 'name': 'bicycle'},
 {'id': 3, 'name': 'car'},
 {'id': 4, 'name': 'motorcycle'},
 {'id': 5, 'name': 'airplane'},
 {'id': 6, 'name': 'bus'},
 {'id': 7, 'name': 'train'},
 {'id': 8, 'name': 'truck'},
 {'id': 9, 'name': 'boat'},
 {'id': 10, 'name': 'traffic light'},
 {'id': 11, 'name': 'fire hydrant'},
 {'id': 13, 'name': 'stop sign'},
 {'id': 14, 'name': 'parking meter'},
 {'id': 15, 'name': 'bench'},
 {'id': 16, 'name': 'bird'},
 {'id': 17, 'name': 'cat'},
 {'id': 18, 'name': 'dog'},
 {'id': 19, 'name': 'horse'},
 {'id': 20, 'name': 'sheep'},
 {'id': 21, 'name': 'cow'},
 {'id': 22, 'name': 'elephant'},
 {'id': 23, 'name': 'bear'},
 {'id': 24, 'name': 'zebra'},
 {'id': 25, 'name': 'giraffe'},
 {'id': 27, 'name': 'backpack'},
 {'id': 28, 'name': 'umbrella'},
 {'id': 31, 'name': 'handbag'},
 {'id': 32, 'name': 'tie'},
 {'id': 33, 'name': 'suitcase'},
 {'id': 34, 'name': 'frisbee'},
 {'id': 35, 'name': 'skis'},
 {'id': 36, 'name': 'snowboard'},
 {'id

## Helper code

In [10]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [11]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
# TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 11) ]
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(4, 5) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(image_np)

---